In [3]:
import numpy as np
import pandas as pd

In [30]:
basic_path_to_dataset = "../../data/ml-1m/"

def read_dataset(basic_path_to_dataset):
    # # read ratings
    filename = "ratings.dat"
    df_ratings = pd.read_csv(basic_path_to_dataset+filename, header=None, sep="::")
    df_ratings.columns = ["UserID", "MovieID", "Rating", "Timestamp"]

    # # read users
    filename = "users.dat"
    df_users = pd.read_csv(basic_path_to_dataset+filename, header=None, sep="::")
    df_users.columns = ["UserID", "Gender", "Age", "Occupation", "Zip-Code"]

    # read movies
    filename = "movies.dat"
    df_movies = pd.read_csv(basic_path_to_dataset+filename, header=None, sep="::")
    df_movies.columns = ["MovieID", "Title", "Genres"]

    return df_ratings, df_users, df_movies

/var/folders/_2/qv661gsd1ws6jbnkjxss7wgw0000gn/T/ipykernel_81386/2481470906.py:15: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_movies = pd.read_csv(basic_path_to_dataset+filename, header=None, sep="::")


In [16]:
df_ratings.head()
df_users.head()

,UserID,Gender,Age,Occupation,Zip-Code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [23]:
df_users["Zip-Code"].unique()

array(['48067', '70072', '55117', ..., '78734', '76006', '14706'],
      dtype=object)